In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd


# phone numbers to check
# Read phone numbers from file and remove country code
with open('USCCphonenumber.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file]
phone_numbers

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.headless = True  # Running in headless mode if preferred
driver = webdriver.Chrome(options=options)


driver.get('https://prepaid.uscellular.com/refill/dashboard')


# Store the results
results = []


try:
    # Wait for and click the cookie reject button if present
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()


    for phone_number in phone_numbers:
        # Wait for the phone number input field to appear and click it to focus
        phone_input_field = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.NAME, "phone"))
        )
       
        phone_input_field.click()  # Focus on the input field


        # Clear the input field in case there's any pre-filled data
        phone_input_field.send_keys(Keys.CONTROL + "a")  # Select all text in input
        phone_input_field.send_keys(Keys.DELETE)  # Delete all selected text


        # Simulate typing the phone number one character at a time
        for char in phone_number:
            phone_input_field.send_keys(char)
            time.sleep(0.1)  # Wait half a second before typing next character


        # Fetch the error message element
        try:
            error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"
            error_message_element = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.XPATH, error_message_xpath))
            )
            if error_message_element.text.strip():  # If there's text in the error element
                error_message = error_message_element.text
        except TimeoutException:
            error_message = "It's a prepaid number" # If no error message appeared within 10 seconds, assume no error
            pass


        # Add the result for this phone number to the results list
        results.append((phone_number, error_message))


        # Clear the input for the next number
        phone_input_field.send_keys(Keys.CONTROL + "a")
        phone_input_field.send_keys(Keys.DELETE)


except Exception as e:
    print(f"An error occurred: {e}")
finally:
    driver.quit()


# Convert the results to a DataFrame
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Message'])
print(df_results)





In [13]:
# Read phone numbers from file and remove country code
with open('USCCphonenumber.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][5:10]
phone_numbers

['4143137825', '2523124309', '2767243709', '4029427871', '3046980011']

In [45]:

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd

# Function to get error code based on the error message
def get_error_code(error_message):
    if 'billed customer' in error_message:
        return 0
    elif 'Phone number not valid' in error_message:
        return 1
    elif 'no longer in service' in error_message:
        return 2
    elif 'No errors, it is a prepaid number' in error_message:
        return 3
    else:
        return -1  # Unknown error message

# Read phone numbers from file and remove country code
with open('USCCphonenumber.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][70:100]
phone_numbers

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.headless = True  
driver = webdriver.Chrome(options=options)

driver.get('https://prepaid.uscellular.com/refill/dashboard')


results = []

try:
    # Reject cookies
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()

    for phone_number in phone_numbers:
        # phone number input field
        phone_input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "phone"))
        )
        phone_input_field.click()  # Focus on the input field

        # Clear the input field first
        phone_input_field.send_keys(Keys.CONTROL + "a")
        phone_input_field.send_keys(Keys.DELETE)

        # Input phone number by numbers one by one
        for char in phone_number:
            phone_input_field.send_keys(char)
            time.sleep(1)  # Small delay between each character

        # Initialize default error message
        error_message = "No errors, it is a prepaid number"
        error_code = get_error_code(error_message)  # Get error code

        # Check for an error message, wait up to 50 seconds
        try:
            error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"
            error_message_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, error_message_xpath))
            )
            if error_message_element.text.strip():  # If there's text in the error element
                error_message = error_message_element.text
                error_code = get_error_code(error_message)  # Update error code
        except TimeoutException:
            # If no error message appeared within 50 seconds, assume no error
            pass


        results.append((phone_number, error_code))

        # next number
        phone_input_field.send_keys(Keys.CONTROL + "a")
        phone_input_field.send_keys(Keys.DELETE)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # close the browser.
    driver.quit()

# Convert the results to a DataFrame 
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
print(df_results)


  Phone Number  Error Code
0   5312800113           0
1   4344208083           0
2   5154992334           2
3   9204723566           0
4   9203924707           0
5   9197270884           0
6   2079075552           0
7   6085471488           2
8   3048886762           0
9   7079544204           3


In [11]:
# df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
# df_results.to_csv('query_result.csv', index=True)  
# print("Results have been saved to query_result.csv")


Results have been saved to query_result.csv


In [37]:
# with open('USCCphonenumber.txt', 'r') as file:
#     phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][10:15]
# phone_numbers

['2623252695', '3196662892', '6033814025', '5158224253', '3192155756']

In [46]:
import os

csv_file_path = 'query_result.csv'

df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    # Append the new results to the existing DataFrame
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))

print(f"Results have been appended to {csv_file_path}")


Results have been appended to query_result.csv


C:\Users\ericluo\AppData\Local\Temp\ipykernel_29416\2355543108.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_existing.append(df_results)
